### Using prompt engineering to explore a dataset

The main idea is to try to capture the company knowledge to better explore the dataset.

The dataset we will use on this notebook is the Netflix shows dataset.

In [54]:
# setting up the environment
import os
import pandas as pd
import openai
import matplotlib.pyplot as plt
import seaborn as sns
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


In [55]:
# Loading the OpenAI API key
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [56]:
# Loading the Netflix dataset
catalog_df = pd.read_csv('./datasets/netflix_titles.csv')

In [154]:
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
"""

In [163]:
prompt = PromptTemplate.from_template(prompt_template)
prompt.format_prompt(context=[], question="what are the longest movies")
def create_prompt(context, question):
    return prompt.format_prompt(context=context, question=question)

In [164]:
agent = create_pandas_dataframe_agent(
    OpenAI(temperature=0),
    catalog_df,
    verbose=True
)

In [236]:
context = """
 Netflix is one of the most popular media and video streaming platforms. 
 They have over 8000 movies or tv shows available on their platform, as of mid-2021, 
 they have over 200M Subscribers globally. This tabular dataset consists of listings 
 of all the movies and tv shows available on Netflix, along with details such as - cast, directors, 
 ratings, release year, duration, etc.
The dataset columns are:
 - director: Director of the Movie
 - cast: Actors involved in the movie / show
 - country: Country where the movie / show was produced
 - date_added: Date it was added on Netflix
 - release_year: Actual Release year of the move / show
 - rating: TV Rating of the movie / show
 - duration: Total Duration - in minutes or number of seasons
"""
agent(create_prompt(context, "explain this dataset"))



> Entering new AgentExecutor chain...
Thought: This dataset contains information about movies and TV shows available on Netflix
Action: python_repl_ast
Action Input: print(df.columns)
Observation: Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

Thought: The dataset contains information about the show, such as the director, cast, country of production, date added to Netflix, release year, rating, duration, and listed in categories.
Final Answer: This dataset contains information about movies and TV shows available on Netflix, such as the director, cast, country of production, date added to Netflix, release year, rating, duration, and listed in categories.

> Finished chain.


{'input': StringPromptValue(text="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer. \nUse three sentences maximum and keep the answer as concise as possible.\n\n Netflix is one of the most popular media and video streaming platforms. \n They have over 8000 movies or tv shows available on their platform, as of mid-2021, \n they have over 200M Subscribers globally. This tabular dataset consists of listings \n of all the movies and tv shows available on Netflix, along with details such as - cast, directors, \n ratings, release year, duration, etc.\nThe dataset columns are:\n - director: Director of the Movie\n - cast: Actors involved in the movie / show\n - country: Country where the movie / show was produced\n - date_added: Date it was added on Netflix\n - release_year: Actual Release year of the move / show\n - rating: TV Rating of the movie / show\n - duration: Total Durati

In [239]:
agent( "what are interesting questions I ask to this dataset")



> Entering new AgentExecutor chain...
Thought: I should look at the columns and think of what I can ask
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')
Thought: I can ask questions about the different columns
Action: python_repl_ast
Action Input: df['type'].value_counts()
Observation: type
Movie      6131
TV Show    2676
Name: count, dtype: int64
Thought: I can ask questions about the different types of movies and tv shows
Final Answer: Interesting questions I can ask about this dataset include: How many movies and TV shows are in the dataset? What are the most common types of movies and TV shows? What countries are represented in the dataset? What is the average release year for movies and TV shows? What is the average rating for movies and TV shows? What is the average duration of movies and TV 

{'input': 'what are interesting questions I ask to this dataset',
 'output': 'Interesting questions I can ask about this dataset include: How many movies and TV shows are in the dataset? What are the most common types of movies and TV shows? What countries are represented in the dataset? What is the average release year for movies and TV shows? What is the average rating for movies and TV shows? What is the average duration of movies and TV shows? What are the most common genres of movies and TV shows?'}

In [240]:
agent("What are the most common genres of movies and TV shows?")



> Entering new AgentExecutor chain...
Thought: I need to look at the 'listed_in' column to find the most common genres.
Action: python_repl_ast
Action Input: df['listed_in'].value_counts()
Observation: listed_in
Dramas, International Movies                          362
Documentaries                                         359
Stand-Up Comedy                                       334
Comedies, Dramas, International Movies                274
Dramas, Independent Movies, International Movies      252
                                                     ... 
Kids' TV, TV Action & Adventure, TV Dramas              1
TV Comedies, TV Dramas, TV Horror                       1
Children & Family Movies, Comedies, LGBTQ Movies        1
Kids' TV, Spanish-Language TV Shows, Teen TV Shows      1
Cult Movies, Dramas, Thrillers                          1
Name: count, Length: 514, dtype: int64
Thought: I now know the most common genres of movies and TV shows.
Final Answer: The most common genres of mo

{'input': 'What are the most common genres of movies and TV shows?',
 'output': 'The most common genres of movies and TV shows are Dramas, International Movies, Documentaries, Stand-Up Comedy, and Comedies, Dramas, International Movies.'}

In [237]:
agent(create_prompt(context, "what are interesting questions I ask to this dataset"))



> Entering new AgentExecutor chain...
Thought: I should think about what kind of questions I can ask about this dataset
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')
Thought: I can ask questions about the columns
Action: python_repl_ast
Action Input: df.describe()
Observation:        release_year
count   8807.000000
mean    2014.180198
std        8.819312
min     1925.000000
25%     2013.000000
50%     2017.000000
75%     2019.000000
max     2021.000000
Thought: I can ask questions about the values in the columns
Final Answer: Interesting questions to ask about this dataset include: what is the distribution of release years? What is the most common country of origin for the movies/shows? What is the most common rating? What is the average duration of the movies/shows?

> Finished chain.


{'input': StringPromptValue(text="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer. \nUse three sentences maximum and keep the answer as concise as possible.\n\n Netflix is one of the most popular media and video streaming platforms. \n They have over 8000 movies or tv shows available on their platform, as of mid-2021, \n they have over 200M Subscribers globally. This tabular dataset consists of listings \n of all the movies and tv shows available on Netflix, along with details such as - cast, directors, \n ratings, release year, duration, etc.\nThe dataset columns are:\n - director: Director of the Movie\n - cast: Actors involved in the movie / show\n - country: Country where the movie / show was produced\n - date_added: Date it was added on Netflix\n - release_year: Actual Release year of the move / show\n - rating: TV Rating of the movie / show\n - duration: Total Durati

In [170]:
agent(create_prompt(context, "what interesting thing you can tell me about the casting"))



> Entering new AgentExecutor chain...
Thought: I should look at the data to see what information I can find about the casting
Action: python_repl_ast
Action Input: df['cast'].describe()
Observation: count                   7982
unique                  7692
top       David Attenborough
freq                      19
Name: cast, dtype: object
Thought: I can tell that David Attenborough is the most frequently cast actor in Netflix movies and tv shows.
Final Answer: David Attenborough is the most frequently cast actor in Netflix movies and tv shows.

> Finished chain.


{'input': StringPromptValue(text="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer. \nUse three sentences maximum and keep the answer as concise as possible.\n\n Netflix is one of the most popular media and video streaming platforms. \n They have over 8000 movies or tv shows available on their platform, as of mid-2021, \n they have over 200M Subscribers globally. This tabular dataset consists of listings \n of all the movies and tv shows available on Netflix, along with details such as - cast, directors, \n ratings, release year, duration, etc.\n\nQuestion: what interesting thing you can tell me about the casting\n"),
 'output': 'David Attenborough is the most frequently cast actor in Netflix movies and tv shows.'}

In [184]:
df = catalog_df[catalog_df.cast.str.contains("David Attenborough", na=False)]
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
757,s758,Movie,Breaking Boundaries: The Science Of Our Planet,Jonathan Clay,"David Attenborough, Johan Rockström",United States,"June 4, 2021",2021,TV-PG,74 min,Documentaries,David Attenborough and scientist Johan Rockstr...
997,s998,TV Show,Life in Color with David Attenborough,NaN,David Attenborough,"Australia, United Kingdom","April 22, 2021",2021,TV-PG,1 Season,"British TV Shows, Docuseries, International TV...","Using innovative technology, this docuseries e..."
1886,s1887,Movie,David Attenborough: A Life on Our Planet,"Alastair Fothergill, Jonnie Hughes, Keith Scholey",David Attenborough,"United States, United Kingdom","October 4, 2020",2020,PG,84 min,Documentaries,"A broadcaster recounts his life, and the evolu..."
3573,s3574,Movie,Our Planet - Behind The Scenes,NaN,David Attenborough,United Kingdom,"August 20, 2019",2019,TV-G,63 min,Documentaries,Years spent recording footage of creatures fro...
3938,s3939,TV Show,Our Planet,NaN,David Attenborough,"United States, United Kingdom","April 5, 2019",2019,TV-PG,1 Season,"Docuseries, Science & Nature TV",Experience our planet's natural beauty and exa...
6094,s6095,TV Show,Africa,NaN,David Attenborough,United Kingdom,"April 28, 2016",2013,TV-PG,1 Season,"British TV Shows, Docuseries, International TV...",This five-part nature series chronicles fascin...
6341,s6342,TV Show,Blue Planet II,NaN,David Attenborough,United Kingdom,"December 3, 2018",2017,TV-G,1 Season,"British TV Shows, Docuseries, Science & Nature TV",This sequel to the award-winning nature series...
6810,s6811,TV Show,Frozen Planet,Alastair Fothergill,David Attenborough,"United Kingdom, United States, Spain, Germany,...","January 28, 2016",2011,TV-PG,1 Season,"British TV Shows, Docuseries, International TV...",Go on a journey through the Arctic and Antarct...
6811,s6812,Movie,Frozen Planet: On Thin Ice,NaN,David Attenborough,"United Kingdom, United States, Spain, Germany,...","January 28, 2016",2011,TV-PG,47 min,Movies,David Attenborough journeys to both polar regi...
6812,s6813,Movie,Frozen Planet: The Epic Journey,NaN,David Attenborough,"United Kingdom, United States, Spain, Germany,...","January 28, 2016",2011,TV-PG,52 min,Movies,Collecting highlights from the epic documentar...


In [188]:
agent("build a new dataset for me where the country columns doesnt have nan values")



> Entering new AgentExecutor chain...
Thought: I need to filter out the nan values
Action: python_repl_ast
Action Input: df[df['country'].notnull()]
Observation:      show_id     type                          title         director  \
0         s1    Movie           Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show                  Blood & Water              NaN   
4         s5  TV Show                   Kota Factory              NaN   
7         s8    Movie                        Sankofa     Haile Gerima   
8         s9  TV Show  The Great British Baking Show  Andy Devonshire   
...      ...      ...                            ...              ...   
8801   s8802    Movie                        Zinzana  Majid Al Ansari   
8802   s8803    Movie                         Zodiac    David Fincher   
8804   s8805    Movie                     Zombieland  Ruben Fleischer   
8805   s8806    Movie                           Zoom     Peter Hewitt   
8806   s8807    Movie            

{'input': 'build a new dataset for me where the country columns doesnt have nan values',
 'output': "df[df['country'].notnull()]"}

In [200]:
context = """
A cast may have many people, the name of each person will appear separated by a comma character.
"""
agent(create_prompt(context, "Create a new dataset for me where if the cast has many people you add a row for each person"))



> Entering new AgentExecutor chain...
Thought: I need to create a new dataset from the existing one
Action: python_repl_ast
Action Input: df.explode('cast')
Observation:      show_id     type                  title         director  \
0         s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1         s2  TV Show          Blood & Water              NaN   
2         s3  TV Show              Ganglands  Julien Leclercq   
3         s4  TV Show  Jailbirds New Orleans              NaN   
4         s5  TV Show           Kota Factory              NaN   
...      ...      ...                    ...              ...   
8802   s8803    Movie                 Zodiac    David Fincher   
8803   s8804  TV Show            Zombie Dumb              NaN   
8804   s8805    Movie             Zombieland  Ruben Fleischer   
8805   s8806    Movie                   Zoom     Peter Hewitt   
8806   s8807    Movie                 Zubaan      Mozez Singh   

                                               

{'input': StringPromptValue(text="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer. \nUse three sentences maximum and keep the answer as concise as possible.\n\nA cast may have many people, the name of each person will appear separated by a comma character.\n\nQuestion: Create a new dataset for me where if the cast has many people you add a row for each person\n"),
 'output': "A new dataset can be created from the existing one by using the `df.explode('cast').reset_index(drop=True)` command, which will separate the cast column into individual rows."}

In [30]:
context.append(agent("what are the longest movies"))



> Entering new AgentExecutor chain...
Thought: I need to find movies with a longer duration
Action: python_repl_ast
Action Input: df[df['duration'] > 90]
Observation: TypeError: '>' not supported between instances of 'str' and 'int'
Thought: I need to convert the duration column to an integer
Action: python_repl_ast
Action Input: df['duration'] = df['duration'].str.replace(' min', '').astype(int)
Observation: ValueError: invalid literal for int() with base 10: '2 Seasons'
Thought: I need to remove the rows with '2 Seasons'
Action: python_repl_ast
Action Input: df = df[df['duration'] != '2 Seasons']
Observation: 
Thought: I need to convert the duration column to an integer
Action: python_repl_ast
Action Input: df['duration'] = df['duration'].str.replace(' min', '').astype(int)
Observation: ValueError: invalid literal for int() with base 10: '1 Season'
Thought: I need to remove the rows with '1 Season'
Action: python_repl_ast
Action Input: df = df[df['duration'] != '1 Season']
Observat

In [31]:
context

[{'input': 'what are the best rated movies',
  'output': "The best rated movies are You Don't Mess with the Zohan, Sex Doll, Immoral Tales, LEGO Ninjago: Masters of Spinjitzu: Day of the Departed, and Motu Patlu: King of Kings."},
 {'input': 'what are the longer movies', 'output': "df[df['duration'] > 90]"}]

In [32]:
catalog_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [45]:
longest_movie_context = """To find the longest movies you need to 
filter by type as Movie and sort the duration in descent order
and pick the top 5
"""

In [46]:
q = prompt.format_prompt(context=longest_movie_context, question="what are the longest movies")

In [47]:
agent(q)



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe by type and sort by duration
Action: python_repl_ast
Action Input: df[df['type'] == 'Movie'].sort_values('duration', ascending=False).head(5)
Observation:      show_id   type                             title          director  \
337     s338  Movie                   Good Luck Chuck     Mark Helfrich   
6915   s6916  Movie                           Hangman     Johnny Martin   
8625   s8626  Movie             Tremors 5:  Bloodline  Don Michael Paul   
2183   s2184  Movie              What Keeps You Alive     Colin Minihan   
721     s722  Movie  Rogue Warfare: Death of a Nation      Mike Gunther   

                                                   cast  \
337   Dane Cook, Jessica Alba, Dan Fogler, Ellia Eng...   
6915  Al Pacino, Karl Urban, Brittany Snow, Joe Ande...   
8625  Michael Gross, Jamie Kennedy, Ernest Ndhlovu, ...   
2183  Hannah Emily Anderson, Brittany Allen, Martha ...   
721   Will Yun Lee,

{'input': StringPromptValue(text="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer. \nUse three sentences maximum and keep the answer as concise as possible.\nTo find the longest movies you need to \nfilter by type as Movie and sort the duration in descent order\nand pick the top 5\n\nQuestion: what are the longest movies\n"),
 'output': "The longest movies are 'Good Luck Chuck', 'Hangman', 'Tremors 5: Bloodline', 'What Keeps You Alive', and 'Rogue Warfare: Death of a Nation'."}

In [48]:
agent("what are the most prolifect directos")



> Entering new AgentExecutor chain...
Thought: I need to find out which directors have the most entries in the dataframe
Action: python_repl_ast
Action Input: df['director'].value_counts()
Observation: director
Rajiv Chilaka                 19
Raúl Campos, Jan Suter        18
Suhas Kadav                   16
Marcus Raboy                  15
Jay Karas                     14
                              ..
Vince Gilligan                 1
Bolanle Austen-Peters          1
Sion Sono                      1
Dennis Rovira van Boekholt     1
Mozez Singh                    1
Name: count, Length: 4373, dtype: int64
Thought: I now know the most prolific directors
Final Answer: The most prolific directors are Rajiv Chilaka, Raúl Campos, Jan Suter, Suhas Kadav, and Marcus Raboy.

> Finished chain.


{'input': 'what are the most prolifect directos',
 'output': 'The most prolific directors are Rajiv Chilaka, Raúl Campos, Jan Suter, Suhas Kadav, and Marcus Raboy.'}

In [49]:
agent("what are the shows with longest run")



> Entering new AgentExecutor chain...
Thought: I need to find the shows with the longest duration
Action: python_repl_ast
Action Input: df.sort_values(by='duration', ascending=False).head()
Observation:      show_id   type                    title        director  \
8185   s8186  Movie               The Aerial   Esteban Sapir   
929     s930  Movie  Dead Again in Tombstone      Roel Reiné   
6725   s6726  Movie            Eyyvah Eyyvah     Hakan Algül   
1629   s1630  Movie                   U-Turn  Derick Cabrido   
7402   s7403  Movie                     Mara     Clive Tonge   

                                                   cast         country  \
8185  Rafael Ferro, Sol Moreno, Jonathan Sandor, Ale...       Argentina   
929   Danny Trejo, Jake Busey, Elysia Rotaru, Dean M...             NaN   
6725  Demet Akbağ, Ata Demirer, Özge Borak, Bican Gü...          Turkey   
1629  Kim Chiu, JM de Guzman, Tony Labrusca, Martin ...     Philippines   
7402  Olga Kurylenko, Javier Botet,

{'input': 'what are the shows with longest run',
 'output': 'The shows with the longest duration are The Aerial, Dead Again in Tombstone, Eyyvah Eyyvah, U-Turn, and Mara.'}

In [52]:
context = """The tv shows with the longest run are the ones with more seasons"""
question = prompt.format_prompt(context=context, question="what are the shows with longest run")

In [53]:
agent(question)



> Entering new AgentExecutor chain...
Thought: I need to find the shows with the most seasons
Action: python_repl_ast
Action Input: df[df['type'] == 'TV Show'].sort_values('duration', ascending=False).head()
Observation:      show_id     type                          title            director  \
8         s9  TV Show  The Great British Baking Show     Andy Devonshire   
67       s68  TV Show              Saved by the Bell                 NaN   
1701   s1702  TV Show          American Horror Story                 NaN   
1998   s1999  TV Show               Call the Midwife  Philippa Lowthorpe   
3541   s3542  TV Show                         Naruto         Hayato Date   

                                                   cast         country  \
8     Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...  United Kingdom   
67    Mark-Paul Gosselaar, Tiffani Thiessen, Mario L...   United States   
1701  Evan Peters, Sarah Paulson, Jessica Lange, Den...   United States   
1998  Vanessa Redgra

{'input': StringPromptValue(text="Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer. \nUse three sentences maximum and keep the answer as concise as possible.\nThe tv shows with the longest run are the ones with more seasons\nQuestion: what are the show with longest run\n"),
 'output': 'The shows with the longest run are The Great British Baking Show, Saved by the Bell, American Horror Story, Call the Midwife, and Naruto.'}

In [151]:
context = """
You a data analyst trying to get the right data from this dataset.

You need to remove from the country columns all values that are not valid countries, like "nan".

The tv show with the longest run are the ones with more seasons.

The country could be a string with more than one country separated by comma. When that happens, split the string\
 using a comma separator.
"""
question = prompt.format_prompt(context=context, question="show the list of tv shows for each country in a table format")

In [152]:
agent(question)



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include TV shows and then group them by country
Action: python_repl_ast
Action Input: df[df['type'] == 'TV Show'].groupby('country').agg('title').reset_index()
Observation: AttributeError: 'SeriesGroupBy' object has no attribute 'reset_index'
Thought: I need to use a different method to group the data
Action: python_repl_ast
Action Input: df[df['type'] == 'TV Show'].groupby('country')['title'].apply(list).reset_index()
Observation:                                 country  \
0                             Argentina   
1                             Australia   
2                     Australia, Canada   
3                                Brazil   
4                                Canada   
..                                  ...   
63                United States, Mexico   
64      United States, Mexico, Colombia   
65  United States, Mexico, Spain, Malta   
66    United States, South Korea, China   
67 

{'input': StringPromptValue(text='Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer. \nUse three sentences maximum and keep the answer as concise as possible.\n\nYou a data analyst trying to get the right data from this dataset.\n\nYou need to remove from the country columns all values that are not valid countries, like "nan".\n\nThe tv show with the longest run are the ones with more seasons.\n\nThe country could be a string with more than one country separated by comma. When that happens, split the string using a comma separator.\n\nQuestion: show the list of tv shows for each country in a table format\n'),
 'output': "The list of TV shows for each country can be found in a table format by using the command `df[df['type'] == 'TV Show'].groupby('country')['title'].apply(list).reset_index()`."}